In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
from datetime import timedelta
import datetime
import re

#### Read in the data. The Nodes column triggers errors because several values are spread out over multiple other columns to the right.

In [2]:
#accre = pd.read_csv('../data/accre-jobs-2020.csv',names=['JOBID', 'ACCOUNT', 'USER', 'REQMEM', 'USEDMEM', 'REQTIME', 'USEDTIME',
       #'NODES', 'CPUS', 'PARTITION', 'EXITCODE', 'STATE', 'NODELIST'])

#MP's previous suggestion for how to read in the file by identifying which columns to include. Resulted in adding additional row with column names as row values

In [3]:
accre = pd.read_csv('../data/accre-jobs-2020.csv', nrows=9000, error_bad_lines=False)

b'Skipping line 3461: expected 13 fields, saw 15\nSkipping line 3462: expected 13 fields, saw 16\nSkipping line 3465: expected 13 fields, saw 15\nSkipping line 3468: expected 13 fields, saw 16\nSkipping line 3472: expected 13 fields, saw 15\nSkipping line 3476: expected 13 fields, saw 16\nSkipping line 3479: expected 13 fields, saw 14\nSkipping line 3482: expected 13 fields, saw 14\nSkipping line 3486: expected 13 fields, saw 14\nSkipping line 3490: expected 13 fields, saw 14\nSkipping line 3494: expected 13 fields, saw 14\nSkipping line 3495: expected 13 fields, saw 16\nSkipping line 3497: expected 13 fields, saw 14\nSkipping line 3498: expected 13 fields, saw 14\nSkipping line 3499: expected 13 fields, saw 14\nSkipping line 3505: expected 13 fields, saw 14\nSkipping line 3517: expected 13 fields, saw 16\nSkipping line 3526: expected 13 fields, saw 15\nSkipping line 3528: expected 13 fields, saw 15\nSkipping line 3529: expected 13 fields, saw 15\nSkipping line 3530: expected 13 fields

In [4]:
accre.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [5]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   JOBID      9000 non-null   object
 1   ACCOUNT    9000 non-null   object
 2   USER       9000 non-null   object
 3   REQMEM     9000 non-null   object
 4   USEDMEM    9000 non-null   object
 5   REQTIME    9000 non-null   object
 6   USEDTIME   9000 non-null   object
 7   NODES      9000 non-null   int64 
 8   CPUS       9000 non-null   int64 
 9   PARTITION  9000 non-null   object
 10  EXITCODE   9000 non-null   object
 11  STATE      9000 non-null   object
 12  NODELIST   9000 non-null   object
dtypes: int64(2), object(11)
memory usage: 914.2+ KB


#### We really only want to look at the "production" partition so rows with other partitions should be removed

In [6]:
#make column names lowercase, per group discussion
accre.columns = accre.columns.str.lower()

In [7]:
#examining range of rows during group discussion
#accre.iloc[3449:3471,:]

#### Remove rows with values other than "production" in the Partition column.
"We really only want to look at the "production" partition so rows with other partitions should be removed"

In [8]:
accre.partition.value_counts()

production        7274
turing            1253
nogpfs             435
sam                 12
debug               10
pascal               8
cgw-capra1           2
maxwell              2
cgw-cqs1             2
cgw-tbi01            1
cgw-dougherty1       1
Name: partition, dtype: int64

In [9]:
#use Boolean mask and save df back to itself.
accre = accre[accre['partition']=='production']

#### Job time is in a format of either d-hh:mm:ss or hh:mm:ss, it needs to be converted to total seconds

In [10]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7274 entries, 0 to 8999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   jobid      7274 non-null   object
 1   account    7274 non-null   object
 2   user       7274 non-null   object
 3   reqmem     7274 non-null   object
 4   usedmem    7274 non-null   object
 5   reqtime    7274 non-null   object
 6   usedtime   7274 non-null   object
 7   nodes      7274 non-null   int64 
 8   cpus       7274 non-null   int64 
 9   partition  7274 non-null   object
 10  exitcode   7274 non-null   object
 11  state      7274 non-null   object
 12  nodelist   7274 non-null   object
dtypes: int64(2), object(11)
memory usage: 795.6+ KB


In [11]:
#pd.to_datetime returned error Unknown string format 13-18:00:00.  Replace the - with :

accre['reqtime'] = accre['reqtime'].str.replace('-',':')
#accre['usedtime'] = accre['usedtime'].str.replace('-',':')

In [13]:
#create a placeholder column in order to view the original and updated values side by side
#accre['reqtime2'] = accre['reqtime']

In [14]:
#accre['reqtime2'] = pd.to_timedelta(accre['reqtime2'])

In [17]:
accre.sample(5)

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
1256,17021284_215,regal,debbra,16384Mn,1229.84M,6:06:00:00,1-22:16:15,1,1,production,0:0,COMPLETED,cn340
2631,17045075_336,winged,lavona,4096Mn,292.47M,4:00:00:00,15:55:49,1,1,production,0:0,COMPLETED,cn1349
777,17016718_96,sunburst,darlyne,8192Mn,972.35M,1:00:00:00,07:07:29,1,1,production,0:0,COMPLETED,cn495
7807,17092513_2,cep,codie,4096Mc,1723.78M,7:00:00:00,2-03:53:28,1,1,production,0:0,COMPLETED,cn1135
3536,17050901_200,winged,lavonda,4096Mn,2132.91M,12:00:00,12:00:22,1,4,production,0:0,COMPLETED,cn430


In [48]:
#Write & test if/else portion of formula. IF the string value in the column is shorter than 9 characters, then it means there is no hour data.
#Fill in the hour value by appending '0-' on the left of the string. This will make all our strings able to parse into an equal number of substrings.

time = '1-22:16:15'

if len(time) <9:
    time = "0-" + time
else: time

In [66]:
#Use regex to select either '-' OR ':' as delimiter for split
#Cast substring to numeric, turning it into an array. Cleaner to do it here than for each individual substring

substring = pd.to_numeric(re.split('[-|:]',time))
substring

array([ 1, 22, 16, 15])

In [74]:
#Separate each element of the array and save to variable representing unit of time
#Add the product of each variable and the multiplier to get the total time in seconds

day = substring[-4]
hours = substring[-3]
minutes = substring[-2]
seconds = substring[-1]

time_in_seconds= day*86400 + hours*3600 + minutes*60 + seconds


In [89]:
sample_df = {'job':['1','2','3','4','5'],'reqtime_test':['23:58:00','2-10:58:00','2-00:00:00','22:18:36','4-02:50:00'],'usedtime_test':['22:19:15','1-23:58:00','1-05:17:04','20:52:47','4-02:50:01']}
sample_df = pd.DataFrame(sample_df,columns=['job','reqtime_test','usedtime_test'])
sample_df

,job,reqtime_test,usedtime_test
0,1,23:58:00,22:19:15
1,2,2-10:58:00,1-23:58:00
2,3,2-00:00:00,1-05:17:04
3,4,22:18:36,20:52:47
4,5,4-02:50:00,4-02:50:01


In [111]:
sample_df['reqtime_seconds'] = sample_df['reqtime_test'].apply(convert_time_to_seconds)

In [112]:
sample_df

,job,reqtime_test,usedtime_test,reqtime_seconds
0,1,23:58:00,22:19:15,86280
1,2,2-10:58:00,1-23:58:00,212280
2,3,2-00:00:00,1-05:17:04,172800
3,4,22:18:36,20:52:47,80316
4,5,4-02:50:00,4-02:50:01,355800


In [106]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job              5 non-null      object
 1   reqtime_test     5 non-null      object
 2   usedtime_test    5 non-null      object
 3   reqtime_seconds  5 non-null      int64 
dtypes: int64(1), object(3)
memory usage: 288.0+ bytes


In [108]:
#Define time_in_seconds()

#def convert_time_to_seconds(time):
    #"""Return an integer value of the time converted into seconds."""
    
    #Identify values not formatted as dhhmmss. Fill in the hour value by appending '0-' on the left of the string. 
    #This will make all our strings able to parse into an equal number of substrings.
    
    #if len(time) <9:
        #time = "0-" + time
   # else: time
    
    #Use regex to select either '-' OR ':' as delimiter for split
    #Cast the list of substrings to numeric, turning it into an array. Cleaner to do it here than for each individual substring
    #time = pd.to_numeric(re.split('[-|:]',time))

    #Separate each element of the array and save to variable representing unit of time
    #day = time[-4]
    #hours = time[-3]
    #minutes = time[-2]
    #seconds = time[-1]

    #Add the product of each unit and it's multiplier to get the total time in seconds
    #time_in_seconds= day*86400 + hours*3600 + minutes*60 + seconds
    
    #return time_in_seconds

In [113]:
#Convert the times into seconds

def convert_time_to_seconds(time):
    """Return an integer value of the time converted into seconds."""
    
    #Identify values not formatted as d-hh:mm:ss. Fill in the hour value by appending '0-' on the left of the string. 
    #This will make all our strings cleanly parse into an equal number of substrings.
    
    if len(time) <9:
        time = "0-" + time
    else: time
    
    #Use regex to select either '-' OR ':' as delimiter for split. The resulting substring list is ordered as [days, hours, minutes, seconds]
    #Cast the list of substrings to numeric, turning it into an array. Cleaner to do it here than for each individual substring
    time = pd.to_numeric(re.split('[-|:]',time))

    #Add the product of each unit and its multiplier to get the total time in seconds
    
    time_in_seconds = time[-4]*86400 + time[-3]*3600 + time[-2]*60 + time[-1]
    
    return time_in_seconds

#### Memory is reported in terms of Megabytes per node (Mc) or Megabytes per core (Mc), this needs to be uniformly converted to Megabytes per core by dividing by the number of cores per node in a job. 

In [17]:
accre.head(10)

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,reqtime2
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,-55 days +02:00:00
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,-55 days +02:00:00
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,-55 days +02:00:00
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,-55 days +02:00:00
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,-55 days +02:00:00
5,16297022,treviso,arabella,122880Mn,65183.02M,13-18:00:00,12-02:31:43,1,24,production,0:0,COMPLETED,cn1443,-55 days +02:00:00
6,16297024,treviso,arabella,122880Mn,64941.85M,13-18:00:00,13-06:54:59,1,24,production,0:0,COMPLETED,cn1531,-55 days +02:00:00
7,16316283,treviso,arabella,122880Mn,68499.51M,13-18:00:00,13-18:00:05,1,24,production,0:0,COMPLETED,cn1535,-55 days +02:00:00
8,16316284,treviso,arabella,122880Mn,67761.41M,13-18:00:00,13-18:00:15,1,24,production,0:0,COMPLETED,cn1464,-55 days +02:00:00
9,16316324,treviso,arabella,122880Mn,68087.66M,13-18:00:00,13-18:00:06,1,24,production,0:0,COMPLETED,cn1473,-55 days +02:00:00


In [31]:
accre.sample(50)

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,reqtime2
6429,17079227,wood,hillery,17000Mn,66.40M,23:58:00,21:03:18,1,1,production,0:0,COMPLETED,cn1349,0 days 23:58:00
2233,17041024,cms,cmspilot,28000Mn,6450.76M,2-00:00:00,1-18:37:21,1,4,production,0:0,COMPLETED,cn1434,-9 days +16:00:00
719,17016718_38,sunburst,darlyne,8192Mn,1960.04M,1-00:00:00,05:46:29,1,1,production,0:0,COMPLETED,cn1439,-5 days +20:00:00
8195,17094298,cms,cmspilot,28000Mn,2748.91M,2-00:00:00,00:13:18,1,4,production,0:0,COMPLETED,cn1354,-9 days +16:00:00
7878,17092515_16,cep,codie,4096Mc,1715.27M,7-00:00:00,3-02:00:44,1,1,production,0:0,COMPLETED,cn422,-30 days +20:00:00
4023,17053757_23,carrot,codie,4096Mc,1724.20M,7-00:00:00,1-23:31:39,1,1,production,0:0,COMPLETED,cn1376,-30 days +20:00:00
6428,17079226,wood,hillery,17000Mn,66.31M,23:58:00,23:58:04,1,1,production,0:0,COMPLETED,cn411,0 days 23:58:00
7490,17091615,cms_lowprio,cmslocal,28000Mn,8405.18M,2-00:00:00,04:50:33,1,4,production,0:0,COMPLETED,cn1481,-9 days +16:00:00
7214,17090680,cms,cmslocal,28000Mn,8304.35M,2-00:00:00,06:17:26,1,4,production,0:0,COMPLETED,cn438,-9 days +16:00:00
3219,17045075_928,winged,lavona,4096Mn,274.99M,4-00:00:00,20:30:54,1,1,production,0:0,COMPLETED,cn1485,-17 days +08:00:00


In [32]:
accre[accre['nodes'].value_counts()

1    7218
2      49
3       7
Name: nodes, dtype: int64

In [ ]:
#MP code shared in chat 10-3
#accre[['exitcode_user', 'exitcode_node']] = accre['exitcode'].str.split(':', expand=True).astype(int)
#new DF only including records with node failures. 
#accre_node_failure = accre.loc[accre['exitcode_node'] != 0]

In [21]:
accre.NODES.nunique()

94

In [22]:
accre.NODES.value_counts()

1     3745514
1       65391
2        2459
3         861
4         579
       ...   
51          1
44          1
43          1
41          1
61          1
Name: NODES, Length: 94, dtype: int64

In [34]:
accre[accre.NODES==1].nunique()

JOBID        3745514
ACCOUNT          172
USER             594
REQMEM           204
USEDMEM       827081
REQTIME          448
USEDTIME      161540
NODES              1
CPUS              23
PARTITION         15
EXITCODE          24
STATE             33
NODELIST         868
dtype: int64

In [36]:
accre[(accre.STATE=='FAILED')|(accre.STATE=='OUT_OF_MEMORY')].shape

(1500, 13)